In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='pearl')
from ipywidgets import Button, HBox, VBox, Box, Layout

# Import YFINANCE
# API that catches market information providing a TICKER
# Used here to get historical assets intel.
import yfinance as yf

In [ ]:
liste = pd.read_csv('full_company_selection_FSB120_PEA.csv',
                               encoding = "ISO-8859-1",
                               delimiter=";"
                              )
liste = liste.iloc[:,1::]

In [ ]:
def better_result(Search,data):
    Search = str(Search)
    res1 = data[data['Company'].str.contains(Search)]
    res2 = data[data['Company'].str.contains(Search.capitalize())]
    res3 = data[data['Company'].str.contains(Search.upper())]
    res4 = data[data['Company'].str.contains(Search.lower())]
    res5 = data[data['ticker'].str.contains(Search.upper())]
    res = pd.concat([res1,res2,res3,res4,res5])
    # dropping duplicte values 
    res.drop_duplicates(subset ="ISIN", 
                     keep = 'first', inplace = True)
    return res

#interact(better_result, Search='Company, Siret,...', data=fixed(liste));



In [ ]:
liste_init = pd.DataFrame(columns=['Marche',
                               'Country of Incorporation',
                               'ISIN',
                               'Company',
                               'Secteur',
                               'Industrie',
                               'ticker'])

stocks_init = pd.DataFrame(columns=['Date',
                                    'ticker',
                               'Open',
                               'High',
                               'Low',
                               'Close',
                               'Volume',
                               'Dividends',
                               'Stock',
                               'Splits'])


In [ ]:
# Get the data for the stock  by specifying the stock ticker, start date, and end date
def grabstock(ticker,stocks):
        # Add suffix for the Paris market
        #ticker = ticker + '.PA'
        yfdata = yf.Ticker(ticker)
        yfdata = yfdata.history(period="max")
        if len(yfdata) == 0:
                print('>> lack '+ticker)
                return stocks
        else:
                yfdata['ticker'] = ticker[0:-3]
                # ignore_index=False permet de preserver les dates
                stocks = stocks.append(yfdata, ignore_index=False)
                return stocks

In [ ]:

def response(change):
    query_str = str(query.value)
    res = better_result(query_str,liste)
    stocks = grabstock(res.ticker.iloc[0]+'.PA',stocks_init)
    with fig1.batch_update():
        intell.value = "Processing data <b>...</b>"
    live_data = pd.DataFrame(stocks)
    ## On calcule la vitesse et l'acceleration
    ## !! pour continuer, executer d'abord ##Extract Structural Datas
    live_augmented_datas =  np.array([live_data.Close.values[0:-2],
                          live_data.Close[1:-1].values-live_data.Close.values[0:-2],
                          live_data.Close[2::].values-2*live_data.Close.values[1:-1]+live_data.Close[0:-2].values]
                                )
    live_augmented_datas = np.transpose(live_augmented_datas)
    live_augmented_datas = pd.DataFrame(live_augmented_datas, columns=['Value','Velocity','Acceleration'])
    # Structure calculation
    sol = np.array([]).reshape(0,2)
    for i in range(0,len(live_augmented_datas)-1):
            B = [live_augmented_datas.loc[i].Acceleration,
                 live_augmented_datas.loc[i+1].Acceleration]
            A = [[live_augmented_datas.loc[i].Velocity,live_augmented_datas.loc[i].Value],
                 [live_augmented_datas.loc[i+1].Velocity,live_augmented_datas.loc[i+1].Value]]
            if np.linalg.det(A) == 0:
                results = [0,0]
                sol = np.vstack((sol,results))
            else:
                results = np.linalg.solve(A,B)
                sol = np.vstack((sol,results))
    structural_datas = pd.DataFrame(sol, columns=['Friction','Stiffness'])
    structural_datas['Delta'] = np.sqrt(structural_datas['Friction']*structural_datas['Friction']/4-structural_datas['Stiffness'])
    live_augmented_datas = live_augmented_datas.join(structural_datas)
    live_augmented_datas = live_augmented_datas.fillna(0)
    live_augmented_datas['Index'] = live_data.index[0:-2]
    special_values = live_augmented_datas[live_augmented_datas.Acceleration.eq(0)]
    special_values = special_values[special_values.Delta>250]
    special_values = special_values.sort_values(by=['Value', 'Index'])
    with fig1.batch_update():
        fig3.data[0].x = live_augmented_datas.Index
        fig3.data[0].y = live_augmented_datas.Value
        intell.value = "Displaying the analysis of <b>"+res.Company.iloc[0]+"</b> company."
        fig1.data[0].header.values = ['Value','Buy/Sell']
        fig1.data[0].cells.values = [special_values.Value, special_values.Velocity.map('{:,.2f} €/j'.format)]
        fig2.data[0].x = live_augmented_datas.Value
        fig2.data[0].y = live_augmented_datas.Velocity
        fig2.data[0].marker.color = live_augmented_datas.index




In [ ]:
trace3 = go.Table(
    header=dict(values=['Value','Buy/Sell'],
                align='center'),
    cells=dict(values=[stocks_init.Close,stocks_init.Date],
               align='center'),
    )

fig1 = go.FigureWidget(data=[trace3],
                    )
fig1.update_layout(title_text = "Detected Eigenvalues",
                   xaxis_title="Stock value in euros",
                   yaxis_title="Stock velocity in euros/day",
                   font=dict(
                        family="Courier New, monospace",
                        size=11,
                        color="#7f7f7f",
                        ),
                  #xaxis_rangeslider_visible=True,
                   autosize=True,
                   #width=400,
                   #height=400,
                 )


#fig1.update_layout(title_text = "PEA-PME + SBF120",
#                  #xaxis_rangeslider_visible=True,
#                 )

trace1 = go.Scatter(x = stocks_init.Date,  
               y = stocks_init.Close,
              # marker=dict(
              #          color = live_augmented_datas['Friction'],
              #         size = abs(live_augmented_datas['Velocity']),
              #          colorscale='Viridis',
              #      ),
               mode = 'lines',
    )

fig3 = go.FigureWidget(data=[trace1],
                    )
fig3.update_layout(title_text = "Stock history",
                   xaxis=dict(
                        rangeslider=dict(
                                visible=False
                                ),
                        type="date"
                        ),
                   xaxis_title="History",
                   yaxis_title="Stock value over time",
                   font=dict(
                        family="Courier New, monospace",
                        size=11,
                        color="#7f7f7f",
                        ),
                   autosize=True,
                   #width=400,
                   #height=400,
                  )

trace2 = go.Scatter( x=stocks_init.Close, 
                     y=stocks_init.High,
                     #color= stocks_init.index,
                     #size=abs(live_augmented_datas.Stiffness),
                     #color_continuous_scale = 'plasma',
                     #marginal_y="violin",
                     #marginal_x="violin",
                     mode = 'markers',
                  )
fig2 = go.FigureWidget(data=[trace2],
                    )

fig2.update_layout(title_text = "Value&Trend over time",
                   xaxis_title="Stock value in euros",
                   yaxis_title="Stock velocity in euros/day",
                   font=dict(
                        family="Courier New, monospace",
                        size=11,
                        color="#7f7f7f",
                        ),
                  #xaxis_rangeslider_visible=True,
                   autosize=True,
                   #width=400,
                   #height=400,
                 )

intell = widgets.HTML(
            value="Hi, welcome to the <b>MoM Analyser</b>.",
            placeholder='',
            description='',
            #layout=Layout(width='500', height='80'),
            )

query = widgets.Text(
            value='',
            placeholder='Company name or ticker',
            description='Search for',
            disabled=False,
            continuous_update=False,
            #layout=Layout(width='500', height='80'),
            )

query.observe(response, names='value')


##VBox([query,intell,fig1,fig3,fig2])
#VBox([HBox([query,intell]),HBox([fig3,fig2,fig1])])

In [1]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

form_item_layout = Layout(
    display='flex',
    flex_flow='column wrap',
    justify_content='space-between'
)

form_items = [
    #Box([query,intell], layout=form_item_layout),
    #Box([query], layout=form_item_layout),
    Box([intell,query,fig3,fig2,fig1], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column wrap',
    #border='solid 2px',
    align_items='stretch',
    width='100%'
))
fig = px.treemap(liste, path=['Secteur','Company'])
fig.show()
form


NameError: name 'intell' is not defined